In [ ]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from time import perf_counter as time
from scipy.interpolate import RegularGridInterpolator
from matplotlib.colors import LightSource
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure
import scipy.ndimage
import pickle
import copy

from pygel3d import hmesh, jupyter_display as jd
import plotly.graph_objs as go
from commons.utils import *
from medial_axis_processing import unfolding, inverse_apply, smoothing
from medial_axis_loader.shared import *
from medial_axis_loader import from_medial_sheet
from commons.display import *

In [ ]:
%load_ext autoreload
%autoreload 2

## Load volumetric data

In [ ]:
# volume_data_path = 'data/vindelev/X19_resampled_uint8.nii'
# niiVol = nib.load(volume_data_path)

In [ ]:
# affine = niiVol.affine
# imgSpacing = niiVol.header['pixdim'][1:4]

# # Downsample
# downsample_factor = 1
# affine[0, 0] = affine[0, 0] * downsample_factor
# affine[1, 1] = affine[1, 1] * downsample_factor
# affine[2, 2] = affine[2, 2] * downsample_factor
# imgSpacing = imgSpacing * downsample_factor

# vol = niiVol.get_fdata().astype('float32') #scipy.ndimage.zoom(niiVol.get_fdata().astype('float32'), 1 / downsample_factor, order=2)
# imgDim = vol.shape

# intMethod = 'linear'  # Options: "linear", "nearest", "slinear", "cubic", "quintic" and "pchip"
# expVal = 0.0  # Value for extrapolation (i.e. values outside volume domain)
# x = np.arange(start=0, stop=imgDim[0], step=1) * imgSpacing[0] + affine[0, 3]
# y = np.arange(start=0, stop=imgDim[1], step=1) * imgSpacing[1] + affine[1, 3]
# z = np.arange(start=0, stop=imgDim[2], step=1) * imgSpacing[2] + affine[2, 3]

# F_vol = RegularGridInterpolator((x, y, z), vol, method=intMethod, bounds_error=False, fill_value=expVal)

In [ ]:
# threshold = 130
# vertices_f, faces_f, _,_ = measure.marching_cubes(vol > threshold , 0.5)
# m = hmesh.Manifold.from_triangles(vertices_f, faces_f)
# hmesh.obj_save("data/input/x19_marching_cubes.obj", m)

In [ ]:
# with open('data/vindelev/F_vol.pck', 'wb') as file_handle:
#     pickle.dump(F_vol, file_handle)
    
# with open('data/vindelev/affine.pck', 'wb') as file_handle:
#     pickle.dump(affine, file_handle)

In [ ]:
with open('data/vindelev/F_vol.pck', 'rb') as file_handle:
    F_vol = pickle.load(file_handle)
    
with open('data/vindelev/affine.pck', 'rb') as file_handle:
    affine = pickle.load(file_handle)

## Load medial axis

In [ ]:
input_mesh = hmesh.load("x19_zoom_mesh_GOODONE.obj")
medial_sheet = hmesh.load("data/vindelev/medial_sheet_x19_zoom2.obj")

# input_mesh.positions()[:] *= 12
# medial_sheet.positions()[:] *= 12

medial_axis = from_medial_sheet.load(input_mesh, medial_sheet)

In [ ]:
display_medial_axis(medial_axis)

In [ ]:
np.mean(medial_axis.rbf)

In [ ]:
medial_axis.rbf[:] = np.mean(medial_axis.rbf)

In [ ]:
smoothing.simple_smooth(medial_axis)

In [ ]:
jd.display(medial_axis.surface)

## Sample voxelized surface intensities along medial axis connection direction

In [ ]:
vertices = medial_axis.surface.positions()
normals = medial_axis.diffs  # points from sheet to surface

vertices_ft = vertices @ affine[0:3, 0:3] + np.transpose(affine[0:3, 3])
normals_ft = normals @ affine[0:3, 0:3]

In [ ]:
distances = np.zeros(medial_axis.outer_points.shape[0])

for i, outer in enumerate(medial_axis.outer_points):
    inner_projection = medial_axis.inner_projections[i]
    diff = outer - inner_projection
    distances[i] = np.linalg.norm(diff)

In [ ]:
num_samples = 2*20
num_vertices = vertices_ft.shape[0]

sample_indices = np.linspace(-1.7, 0.7, num_samples)[:, None] * distances[None, :]

samples_ft = vertices_ft[None, :, :] + normals_ft[None, :, :] * sample_indices[:, :, None]

samples_ft_reshaped = samples_ft.reshape(-1, 3)

intensities = F_vol(samples_ft_reshaped)

intensities_reshaped = intensities.reshape(num_samples, num_vertices)
max_intensities = np.max(intensities_reshaped, axis=0)

In [ ]:
# Convert to RGB color
# rgbColor = np.transpose(np.tile(max_intensities.astype('uint8'),(3,1))) / 256
# display_mesh_vertex_colors(medial_axis.surface, vertex_colors=rgbColor) #, save_html="results/vindelev/voxelized")

## Reconstruct coin based on intesities

In [ ]:
intensities = max_intensities / 256
intensities = (intensities - np.min(intensities)) / (np.max(intensities) - np.min(intensities))

In [ ]:
len = np.mean(medial_axis.rbf)

In [ ]:
pos = medial_axis.surface.positions()
new_pos = np.copy(pos)

for i, outer in enumerate(medial_axis.outer_points):
    inner_projection = medial_axis.inner_projections[i]
    diff = outer - inner_projection
    diff /= np.linalg.norm(diff)
    
    new_pos[i] = inner_projection + (diff * (intensities[i] * len))

In [ ]:
medial_axis.outer_points[:] = new_pos
medial_axis.surface.positions()[:] = new_pos

In [ ]:
# display_mesh(medial_axis.surface)

In [ ]:
hmesh.obj_save("results/vindelev/x19_test.obj", medial_axis.surface)

## Unfold reconstructed coin

In [ ]:
unfolded_sheet = unfolding.get_unfolded_sheet_positions(medial_axis)

updated_sheet = hmesh.Manifold(medial_axis.sheet)
updated_sheet.positions()[:] = unfolded_sheet
# hmesh.flip_orientation(updated_sheet)

In [ ]:
inverse_apply.inverse_apply_medial_axis(medial_axis, unfolded_sheet, [])

In [ ]:
smooth(medial_axis.surface, max_iter=7)

In [ ]:
# display_mesh_vertex_colors(medial_axis.surface, vertex_colors=rgbColor, save_html="results/vindelev/voxelized_unfolded")

In [ ]:
# display_mesh(medial_axis.surface)

In [ ]:
hmesh.obj_save("results/vindelev/x19_test_unfolded.obj", medial_axis.surface)

In [ ]:
display_medial_axis(medial_axis)